In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import tensorflow as tf
import numpy as np
if __name__ == "__main__":
    sys.path.append("../../")
from notebooks_src.metrics.mAP import calc_batch_metrics, EpochMetrics, calc_ap_one_class

importing Jupyter notebook from ../../notebooks_src/metrics/mAP.ipynb
importing Jupyter notebook from ../../notebooks_src/postprocessing/utils.ipynb
importing Jupyter notebook from ../../notebooks_src/box_encode_decode/ssd/encode.ipynb
importing Jupyter notebook from ../../notebooks_src/box_encode_decode/ssd/make_anchors_orig.ipynb
box_encode_decode_configs
importing Jupyter notebook from ../../notebooks_src/configs/box_encode_decode_configs.ipynb
callbacks_configs
importing Jupyter notebook from ../../notebooks_src/configs/callbacks_configs.ipynb
fit_configs
importing Jupyter notebook from ../../notebooks_src/configs/fit_configs.ipynb
labels_configs
importing Jupyter notebook from ../../notebooks_src/configs/labels_configs.ipynb
load_data_configs
importing Jupyter notebook from ../../notebooks_src/configs/load_data_configs.ipynb
losses_configs
importing Jupyter notebook from ../../notebooks_src/configs/losses_configs.ipynb
metrics_configs
importing Jupyter notebook from ../../notebook

In [ ]:
def fit(model, generator, val_generator,num_epochs, loss_func, opt):
    with tf.Session() as sess:


        loss_tensor, label  = get_loss_tensor(loss_func, model, generator)
        train_step = opt.minimize(loss_tensor)
        sess.run(tf.global_variables_initializer())
        input_ = model.input
        for ep in range(num_epochs):
            losses = []
     
            steps_per_epoch= generator.num_ims / generator.batch_size

            for step in range(steps_per_epoch):
                im, boxes = generator.next()
                _,cur_loss = sess.run([train_step, loss_tensor],feed_dict={input_:im,label:boxes})
                losses.append(cur_loss)     
                print "loss: %8.4f"% cur_loss
            tr_loss = sum(losses) / len(losses)
            #tr_loss = get_epoch_loss(generator, model, train_step, loss_tensor,sess,label)
            #val_loss = get_val_epoch_loss(generator, model,loss_tensor,sess,label)
            
            val_mAP, val_APs = get_epoch_accuracy(val_generator, model, sess, input_)
            tr_mAP, tr_APs = get_epoch_accuracy(generator, model, sess, input_)
            
            print "tr_loss: ", tr_loss
            #print "val_loss: ", val_loss
            print "val_mAP: ", val_mAP
            print "tr_mAP: ", tr_mAP
                
                

In [3]:
def get_val_epoch_loss(generator,model,loss_tensor,sess,label):
    losses = []
    input_ = model.input
    steps_per_epoch= generator.num_ims / generator.batch_size

    for step in range(steps_per_epoch):
        im, boxes = generator.next()
        cur_loss = sess.run(loss_tensor,feed_dict={input_:im,label:boxes})
        losses.append(cur_loss)     
        print "loss: %8.4f"% cur_loss
    return sum(losses) / len(losses)

In [2]:
def get_epoch_loss(generator,model, train_step, loss_tensor,sess,label):
    losses = []
    input_ = model.input
    steps_per_epoch= generator.num_ims / generator.batch_size

    for step in range(steps_per_epoch):
        im, boxes = generator.next()
        _,cur_loss = sess.run([train_step, loss_tensor],feed_dict={input_:im,label:boxes})
        losses.append(cur_loss)     
        print "loss: %8.4f"% cur_loss
    return sum(losses) / len(losses)

In [ ]:
def get_epoch_accuracy(generator, model, sess,input_):
    epm = EpochMetrics()
    batch_accuracy_tensors, y_true = get_batch_accuracy_tensors(calc_batch_metrics, model, generator)
    steps_per_epoch = generator.num_ims / generator.batch_size
    for step in range(steps_per_epoch):
        im, boxes = generator.next()
        updated_metrics = sess.run(batch_accuracy_tensors, feed_dict={y_true:boxes, input_:im})
        epm.update_metrics(*updated_metrics)

    final_metrics = epm.get_final_metrics()
    aps_voc12, placeholders= calc_ap_one_class()
    all_aps12 = {}


    for c in final_metrics[0].keys():
        placefillers = [d[c] for d in final_metrics]
        all_aps12[c] = sess.run(aps_voc12, feed_dict = dict(zip(placeholders, placefillers)) )


    mAP = np.mean(all_aps12.values())
    return mAP, all_aps12

In [ ]:
def get_y_true_y_preds_tensors(model, batch_size, label_shape):
    output_tensors = model.outputs
        
    label_batch_shape = tuple([batch_size] + list(label_shape))
        
        
    label_tensor = tf.placeholder(tf.float32,shape=label_batch_shape, name="label")
    return label_tensor, output_tensors

In [ ]:
def get_loss_tensor(loss_func, model, generator):
        batch_size = generator.batch_size
        
        
        y_true, y_preds = get_y_true_y_preds_tensors(model, batch_size,generator.data.labels.shape[1:])
        loss_tensor = loss_func(y_true, y_preds)

        return loss_tensor, y_true

In [ ]:
def get_batch_accuracy_tensors(acc_func, model,generator):
        batch_size = generator.batch_size
        y_true, y_preds = get_y_true_y_preds_tensors(model, batch_size,generator.data.labels.shape[1:])
        batch_metrics = calc_batch_metrics(y_true, y_preds)
        return batch_metrics, y_true
        
    